In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [7]:
from datetime import timedelta
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
import math
import os
from matplotlib.pyplot import figure
from datetime import datetime
import warnings
import ipywidgets as widgets
from dateutil.relativedelta import relativedelta

warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None

from_user = 21
to_user = 22
src_path = "C:/Users/12sha/Documents/thesislocation/Data/Final Example Results"

# fetch user vs train test months

usern_mnth_df = pd.read_csv("C:/Users/12sha/Documents/thesislocation/Data/user_vs_traintestmonth.csv", sep = "\t")
usern_mnth_df = usern_mnth_df.drop(['Unnamed: 0'], axis=1)

In [13]:
#Cosine Similarity

# Find distance between two lan:lon points in meters
def meters(lat1, lon1, lat2, lon2):
    R = 6378.137  # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(lat1 * math.pi / 180) * math.cos(
        lat2 * math.pi / 180) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    return d * 1000  # meters

def dotproduct(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def length(v):
    return math.sqrt(dotproduct(v, v))

#def find_similarity(pred_prob, act_prob_all):
#    return math.acos(dotproduct(pred_prob, act_prob_all) / (length(pred_prob) * length(act_prob_all)))

def find_similarity(pred_prob, act_prob_all):
    result = 1 - spatial.distance.cosine(pred_prob, act_prob_all)
    return result

# calculate prediction parameters
def check_pred(hour, indx_row):
    global predic_df
    global hrywghts_df
    global similarity_arr

    pred_prob = predic_df['PredProbability'].values
    act_prob_all = [0] * len(pred_prob)
    curr_date = hrywghts_df.loc[indx_row, 'Date']
    state_wght = hrywghts_df.loc[indx_row, str(hour)]

    jmp_dat = False
    if hour == 23:
        jmp_dat = True

    # check if there are points found in next hour
    hrywghts_day_df = hrywghts_df.loc[hrywghts_df['Date'] == curr_date]
    hrywghts_day_df = hrywghts_day_df.reset_index(drop=True)
    for k in range(0, len(hrywghts_day_df)):
        if jmp_dat == True:
            if k + 1 < len(hrywghts_day_df):
                next_hour = 0
                row = k + 1
            else:
                break
        else:
            next_hour = hour + 1
            row = k

        if hrywghts_day_df.loc[row, str(next_hour)] != 0:

            true_lat = hrywghts_day_df['AvgLat'][row]
            true_lon = hrywghts_day_df['AvgLon'][row]

            for i in range(0, len(predic_df)):
                pred_lat = predic_df.loc[i, 'Latitude']
                pred_lon = predic_df.loc[i, 'Longitude']

                if meters(true_lat, true_lon, pred_lat, pred_lon) <= state_d_thrhld:
                    act_prob_all[i] = hrywghts_day_df.loc[k, str(next_hour)]
                    break

#     #if the actual_prob_all has all 0's, this means the prediction is incorrect and hence similarity is 0.
#     if np.mean(act_prob_all) == 0:
#         similarity = 90
#     else:
#         similarity = find_similarity(pred_prob, act_prob_all)
    similarity = find_similarity(pred_prob, act_prob_all)
    similarity_arr = np.append(similarity_arr, similarity)
    predic_df['ActProbability'] = act_prob_all
    predic_df['Similarity'] = similarity


def predict():
    global trained_model_df
    global hrywghts_df
    global predic_df

    file_name = "Similarity.csv"
    file = dest_predicted_dir + file_name

    # remove if the file already exists
    try:
        os.remove(file)
    except OSError:
        pass

    for j in range(0, len(hrywghts_df)):
        # If user has data for hours
        for h in range(0, 24):
            if hrywghts_df.loc[j, str(h)] != 0:
                hour = h
                new_lat = hrywghts_df['AvgLat'][j]
                new_lon = hrywghts_df['AvgLon'][j]
                # current state probabilities                        
                curr_state_prob = hrywghts_df.loc[j, str(h)]

                for i in range(0, len(trained_model_df)):

                    trn_lat = trained_model_df['AvgLat'][i]
                    trn_lon = trained_model_df['AvgLon'][i]
                    predic_df = pd.DataFrame()

                    if meters(trn_lat, trn_lon, new_lat, new_lon) <= state_d_thrhld:

                        cluster_id = trained_model_df['StateId'][i]

                        # next hour prediction probabilities
                        jmp_dat = False
                        if hour == 23:
                            jmp_dat = True

                        if jmp_dat == True:
                            # check if there exists a next row
                            if i+1 < len(trained_model_df):
                                from_col_no = 5
                                to_col_no = from_col_no + trained_model_df['StateId'].nunique()
                                predic_df = trained_model_df.iloc[i + 1:i + 2, from_col_no:to_col_no]
                            else:
                                break
                        else:
                            from_col_no = trained_model_df['StateId'].nunique() * (hour + 1) + 5
                            to_col_no = from_col_no + trained_model_df['StateId'].nunique()
                            predic_df = trained_model_df.iloc[i:i + 1, from_col_no:to_col_no]

                        predic_df = (predic_df * curr_state_prob).T

                        predic_df['StateId'] = cluster_id
                        predic_df['PredState'] = predic_df.index
                        predic_df['PredState'] = predic_df['PredState'].map(lambda x: x.split('-', 2)[-1])

                        predic_df.columns = ['PredProbability', 'StateId', 'PredState']

                        # predic_df = predic_df.sort_values('Probability', ascending=False).head(10)
                        predic_df = predic_df.sort_values('PredProbability', ascending=False)
                        predic_df['DateHour'] = str(hrywghts_df['Date'][j]) + " " + str(hour)
                        predic_df['Address'] = 0
                        predic_df['Latitude'] = 0.0
                        predic_df['Longitude'] = 0.0
                        predic_df = predic_df.reset_index(drop=True)

                        for k in range(0, len(predic_df)):
                            # import pdb; pdb.set_trace()
                            clus_to_find = int(float(predic_df['PredState'][k]))
                            add = trained_model_df.loc[(trained_model_df['StateId'] == clus_to_find), 'Address'].values[
                                0]
                            lat = trained_model_df.loc[(trained_model_df['StateId'] == clus_to_find), 'AvgLat'].values[
                                0]
                            lon = trained_model_df.loc[(trained_model_df['StateId'] == clus_to_find), 'AvgLon'].values[
                                0]

                            predic_df.loc[k, 'Address'] = add
                            predic_df.loc[k, 'Latitude'] = lat
                            predic_df.loc[k, 'Longitude'] = lon

                        # if prediction was made, calculate prediction parameters
                        check_pred(hour, j)

                        predic_df.to_csv(file, mode='a')
                        break


# -------------------------------------------------------------------------------------------------------
state_d_thrhld = 200
cos_sim = []

for i in range (from_user, to_user):
    
    user = "{0:0=3d}".format(usern_mnth_df.loc[i, 'User'])
    train_month = str(usern_mnth_df.loc[i, 'TrainMonth'])
    test_month = str(usern_mnth_df.loc[i, 'TestMonth'])
    
    tobepredicted_df = pd.DataFrame()
    predic_df = pd.DataFrame()

    # hourly weights file
    hrly_wghts_file = src_path + "/User " + user + "/" + test_month + "/hourlyweights/hourlyweights.csv"
    # trained model
    trained_model_file = src_path + "/User " + user + "/" + train_month + "/markovchains/final.csv"
    # predicted file
    dest_predicted_dir = src_path + "/User " + user + "/" + test_month + "/predict/"

    similarity_arr = []
    
     # check if the path is found
    if ((not os.path.exists(src_path + "/User " + user + "/" + test_month)) or
       not os.path.exists(src_path + "/User " + user + "/" + train_month)):
        print("For user: " + user + " test and training path not found." +
             " Check if you have files inside: <<" + src_path + "/User " + user + "/" + test_month + "/staypoints/staypoints.csv"
             ">> and <<" + src_path + "/User " + user + "/" + train_month + "/markovchains/final.csv>>")
        continue
    
    # check if the file is found
    if ((not os.path.isfile(trained_model_file)) or
        (not os.path.isfile(hrly_wghts_file))):
        print("For user: " + user + " test and training files not found.")
        continue

    trained_model_df = pd.read_csv(trained_model_file, header=0)
    hrywghts_df = pd.read_csv(hrly_wghts_file, header=0, sep='\t')

    predict()

    # save similarity results
    if similarity_arr:
        similarity_arr = similarity_arr[np.logical_not(np.isnan(similarity_arr))]
        sim_mean = np.mean(similarity_arr)
        text_file = dest_predicted_dir + "Similarity mean " + str(sim_mean) + " .txt"
        f = open(text_file, "w+")
        print("User: " + user + ". Cosine Similarity is: " + str(sim_mean) + "degrees")

        cos_sim.append(sim_mean)
if cos_sim:    
    print("mean cosine similarity: " + str(sum(cos_sim)/len(cos_sim)))